# CreateMiniDump Hacktool
Detects the use of CreateMiniDump hack tool used to dump the LSASS process memory for credential extraction on the attacker's machine

## Rule Content
```
- action: global
  title: CreateMiniDump Hacktool
  id: 36d88494-1d43-4dc0-b3fa-35c8fea0ca9d
  description: Detects the use of CreateMiniDump hack tool used to dump the LSASS
    process memory for credential extraction on the attacker's machine
  author: Florian Roth
  references:
  - https://ired.team/offensive-security/credential-access-and-credential-dumping/dumping-lsass-passwords-without-mimikatz-minidumpwritedump-av-signature-bypass
  date: 2019/12/22
  tags:
  - attack.credential_access
  - attack.t1003
  falsepositives:
  - Unknown
  level: high
- logsource:
    category: process_creation
    product: windows
  detection:
    selection1:
      Image|contains: \CreateMiniDump.exe
    selection2:
      Imphash: 4a07f944a83e8a7c2525efa35dd30e2f
    condition: 1 of them
- logsource:
    product: windows
    service: sysmon
  detection:
    selection:
      EventID: 11
      TargetFileName|contains: '*\lsass.dmp'
    condition: 1 of them

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:*\\CreateMiniDump.exe* OR hash_imphash:"4a07f944a83e8a7c2525efa35dd30e2f")')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='(event_id:"11" AND TargetFileName.keyword:*\\lsass.dmp*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()